# Cleaning: BJH026


Couldn't save hippocampal contacts. Saved right amygdala, ofc, sgacc, insula, dlpfc by taking out

'GR14', 'GR13', 'HL1', 'HL3', 'HL2', 'JL2', 'IL3', 'IL4', 'JL1', 'HL5', 'EL8', 'GL1', 'IL2', 'IL1', 'JL3', 'GL3', 'GL2', 'IL6', 'IL5', 'JL5', 'JL4', 'JL8', 'JL7', 'JL6', 'EL2', 'EL1', 'EL5', 'EL4', 'EL3', 'EL7', 'EL6', 'FL4', 'FL5', 'FL3', 'FL2', 'FL1', 'EL10', 'EL9', 'FL10', 'FL9', 'FL8', 'FL7', 'FL6', 'HL8', 'HL7', 'HL6', 'HL4', 'GL8', 'GL7', 'GL6', 'GL5', 'GL4'

### Prep

In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
%matplotlib qt5

In [2]:
# helper functions#

def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])
        
def find_bad_trial_times(idx, srate, trial_begins, trial_lengths):
    ''' takes and index of trial lengths, beginnings and the sampling rate and returns the the start time and and
    duration of the times to annotae as BAD '''
    start_time = trial_begins[0][idx]/srate
    duration = trial_lengths[0][idx]/srate
    
    return start_time, duration        

In [3]:
## Prep paths ##

subject = 'BJH026'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [4]:
## Load Data ##

raw_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_ieeg.fif")

Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_ieeg-1.fif...
    Range : 2086000 ... 2920199 =   1043.000 ...  1460.100 secs
Ready.


In [6]:
## Filtering ##

# load data #
raw_fif.load_data()

# highpass filter @ 1 and lowpass @ 200 #
filtered_data = raw_fif.filter(l_freq=1, h_freq=150) # skip this step for TF analysis, always do this on the least processed data, or do .1 to 250

# remove 60hz line noise #
freqs = (60, 120, 180, 240)
filtered_notch_data = filtered_data.notch_filter(freqs=freqs) 


Reading 0 ... 2920199  =      0.000 ...  1460.100 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 150.00 Hz
- Upper transition bandwidth: 37.50 Hz (-6 dB cutoff frequency: 168.75 Hz)
- Filter length: 6601 samples (3.300 sec)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    5.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   12.5s


Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 13201 samples (6.601 sec)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    4.8s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   12.0s


In [5]:
## Load/Save Filtered Data ##

# save #
# filtered_notch_data.save(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_ieeg.fif", overwrite = True)

# load #
filtered_notch_data = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_ieeg.fif")
events = mne.find_events(filtered_notch_data, output='step', consecutive = False, stim_channel='STI')

Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_ieeg-1.fif...
    Range : 2086000 ... 2920199 =   1043.000 ...  1460.100 secs
Ready.
480 events found
Event IDs: [0 1]


In [6]:
filtered_notch_data.info['ch_names']

['EMPTY',
 'EMPTY_2',
 'EMPTY_3',
 'EMPTY_4',
 'REF1',
 'REF2',
 'AL1',
 'AL2',
 'AL3',
 'AL4',
 'AL5',
 'AL6',
 'AL7',
 'AL8',
 'AL9',
 'AL10',
 'AL11',
 'AL12',
 'AL13',
 'AL14',
 'AL15',
 'AL16',
 'BL1',
 'BL2',
 'BL3',
 'BL4',
 'BL5',
 'BL6',
 'BL7',
 'BL8',
 'BL9',
 'BL10',
 'BL11',
 'BL12',
 'BL13',
 'BL14',
 'BL15',
 'BL16',
 'CL1',
 'CL2',
 'CL3',
 'CL4',
 'CL5',
 'CL6',
 'CL7',
 'CL8',
 'DL1',
 'DL2',
 'DL3',
 'DL4',
 'DL5',
 'DL6',
 'DL7',
 'DL8',
 'DL9',
 'DL10',
 'DL11',
 'DL12',
 'DL13',
 'DL14',
 'DL15',
 'DL16',
 'EMPTY_63',
 'EMPTY_64',
 'EL1',
 'EL2',
 'EL3',
 'EL4',
 'EL5',
 'EL6',
 'EL7',
 'EL8',
 'EL9',
 'EL10',
 'FL1',
 'FL2',
 'FL3',
 'FL4',
 'FL5',
 'FL6',
 'FL7',
 'FL8',
 'FL9',
 'FL10',
 'GL1',
 'GL2',
 'GL3',
 'GL4',
 'GL5',
 'GL6',
 'GL7',
 'GL8',
 'HL1',
 'HL2',
 'HL3',
 'HL4',
 'HL5',
 'HL6',
 'HL7',
 'HL8',
 'IL1',
 'IL2',
 'IL3',
 'IL4',
 'IL5',
 'IL6',
 'IL7',
 'IL8',
 'IL9',
 'IL10',
 'IL11',
 'IL12',
 'JL1',
 'JL2',
 'JL3',
 'JL4',
 'JL5',
 'JL6',
 'JL

## Cleaning Neural Data

#### Bad Channels

Epileptic channels from my cleaning: 

Noisy channels: 

In [7]:
# Remove Channels from filtered data

# remove non seeg electrodes electrodes #
string_lst = ['EMPTY', 'REF', 'ekg']

bad_channels_reg = [re.findall(r"(?=("+'|'.join(string_lst)+r"))", name, re.IGNORECASE) for name in filtered_notch_data.info['ch_names']]
bad_channels_index = [i for i,x in enumerate(bad_channels_reg) if  x]
bad_channels = [filtered_notch_data.info['ch_names'][i] for i in bad_channels_index]

filtered_notch_data.info['bads'].extend(bad_channels)  

# # remove epileptic electrodes
filtered_notch_data.info['bads'].extend(['GR14', 'GR13', 'HL1', 'HL3', 'HL2', 'JL2', 'IL3', 'IL4', 'JL1', 'HL5', 'EL8', 'GL1', 'IL2', 'IL1', 'JL3', 'GL3', 'GL2', 'IL6', 'IL5', 'JL5', 'JL4', 'JL8', 'JL7', 'JL6', 'EL2', 'EL1', 'EL5', 'EL4', 'EL3', 'EL7', 'EL6', 'FL4', 'FL5', 'FL3', 'FL2', 'FL1', 'EL10', 'EL9', 'FL10', 'FL9', 'FL8', 'FL7', 'FL6', 'HL8', 'HL7', 'HL6', 'HL4', 'GL8', 'GL7', 'GL6', 'GL5', 'GL4'])  

# remove out of brain electrodes
## TODO hasn't been localized

# remove scalp and dc channels
scalp_dc_channels = ['FP1',
 'F3',
 'C3',
 'P3',
 'O1',
 'FP2',
 'F4',
 'C4',
 'P4',
 'O2',
 'F7',
 'T7',
 'P7',
 'F8',
 'T8',
 'P8',
 'F9',
 'F10',
 'FPZ',
 'FZ',
 'CZ',
 'PZ',
 'OZ']

filtered_notch_data.info['bads'].extend(scalp_dc_channels)
filtered_notch_data

<Raw | BJH026_notched_filtered_ieeg.fif, 257 x 2920200 (1460.1 s), ~261 kB, data not loaded>

In [8]:
# Remove Channels from filtered data

# remove non seeg electrodes electrodes #
string_lst = ['EMPTY', 'REF', 'ekg']

bad_channels_reg = [re.findall(r"(?=("+'|'.join(string_lst)+r"))", name, re.IGNORECASE) for name in raw_fif.info['ch_names']]
bad_channels_index = [i for i,x in enumerate(bad_channels_reg) if  x]
bad_channels = [raw_fif.info['ch_names'][i] for i in bad_channels_index]

raw_fif.info['bads'].extend(bad_channels)  

# # remove epileptic electrodes
raw_fif.info['bads'].extend(['GR14', 'GR13', 'HL1', 'HL3', 'HL2', 'JL2', 'IL3', 'IL4', 'JL1', 'HL5', 'EL8', 'GL1', 'IL2', 'IL1', 'JL3', 'GL3', 'GL2', 'IL6', 'IL5', 'JL5', 'JL4', 'JL8', 'JL7', 'JL6', 'EL2', 'EL1', 'EL5', 'EL4', 'EL3', 'EL7', 'EL6', 'FL4', 'FL5', 'FL3', 'FL2', 'FL1', 'EL10', 'EL9', 'FL10', 'FL9', 'FL8', 'FL7', 'FL6', 'HL8', 'HL7', 'HL6', 'HL4', 'GL8', 'GL7', 'GL6', 'GL5', 'GL4'])  

# remove out of brain electrodes
## TODO hasn't been localized

# remove scalp and dc channels
scalp_dc_channels = ['FP1',
 'F3',
 'C3',
 'P3',
 'O1',
 'FP2',
 'F4',
 'C4',
 'P4',
 'O2',
 'F7',
 'T7',
 'P7',
 'F8',
 'T8',
 'P8',
 'F9',
 'F10',
 'FPZ',
 'FZ',
 'CZ',
 'PZ',
 'OZ']

raw_fif.info['bads'].extend(scalp_dc_channels)
raw_fif

<Raw | BJH026_raw_ieeg.fif, 257 x 2920200 (1460.1 s), ~261 kB, data not loaded>

#### Bad Epochs from noisy data



In [9]:
# # Annotate Noisy Epochs from Bob's cleaning

# read from csv from interactive session
bad_annots = mne.read_annotations('../ieeg/saved_annotations.fif')   


#### Bad epochs from bad trial data

In [10]:
## quality check trials ##

# calculate trial onsets and offsets
sti_raw = filtered_notch_data.get_data(picks = ['STI'])
lengths, positionsm, val  = rle(sti_raw[0])

# lengths, beginnings, endings, for trials
trial_lengths = [lengths[x] for x in np.where(val == 1)]
trial_begins = [positionsm[x] for x in np.where(val == 1)]
trial_ends = trial_lengths[0] + trial_begins[0]
trial_baseline = trial_begins[0] - 1000

# lengths, beginnings, endings, for itis
iti_lengths = [lengths[x] for x in np.where(val == 0)]
iti_begins = [positionsm[x] for x in np.where(val == 0)]
iti_ends = iti_begins[0] + iti_lengths[0]

# save trial lengths for later
np.save(f"{preproc_data_dir}/{subject}_trial_lengths.npy", trial_lengths)

In [11]:
# exclude bad trials/apochs from r behavioral analysis (paused trials and no biscuits)

# load bad trial data
bad_trials = np.genfromtxt(f"{raw_data_dir}/behave/{subject}_bad_trials.csv", delimiter = ',', skip_header = 1)

onsets = []
durations = []
for bad in bad_trials:
    start, dur = find_bad_trial_times(int(bad), raw_fif.info['sfreq'], trial_begins, trial_lengths)
    onsets.append(start)
    durations.append(dur)

# update descriptions
descriptions = ['bad'] * len(durations)


In [12]:
## exclude all bad epochs from both bad trials and investigating neural data ##

# # combine with bad trial data #
bad_annots.append(onsets, durations, descriptions)

# # # apply to filtered data #
filtered_notch_data.set_annotations(bad_annots)

# # # apply to not filtered data #
raw_fif.set_annotations(bad_annots)

<Raw | BJH026_raw_ieeg.fif, 257 x 2920200 (1460.1 s), ~261 kB, data not loaded>

## Visualizing Neural Data

next step, working on filtering out the high frequency noise

then look at Marks script to come up with some plot of every trial, sort longest to shortest, with theta power as the color tiome as x, trial on y

In [13]:
# plot for cleaning #
filtered_notch_data.drop_channels(filtered_notch_data.info['bads'])
filtered_notch_data.plot(events=events, color='b', bad_color = 'red', n_channels = 230, clipping = None, event_color = 'r')

Using qt as 2D backend.


Channels marked as bad:
['GL1', 'IL2', 'IL1', 'JL3', 'GL3', 'GL2', 'IL6', 'IL5', 'JL5', 'JL4', 'JL8', 'JL7', 'JL6', 'EL2', 'EL1', 'EL5', 'EL4', 'EL3', 'EL7', 'EL6', 'FL4', 'FL5', 'FL3', 'FL2', 'FL1', 'EL10', 'EL9', 'FL10', 'FL9', 'FL8', 'FL7', 'FL6', 'HL8', 'HL7', 'HL6', 'HL4', 'GL8', 'GL7', 'GL6', 'GL5', 'GL4']


In [13]:
# plot psd of all channels #

filtered_notch_data.plot_psd()

In [ ]:
# plot psd of bad channels #

filtered_notch_data.plot_psd(picks = filtered_notch_data.info['bads'])

In [15]:
# # From interactive sessions
# interactive_annot = filtered_notch_data.annotations
# for x in range(0, len(interactive_annot)):
#     print(interactive_annot[x])
    
    
# filtered_notch_data.annotations.save('../ieeg/saved_annotations.fif', overwrite = True)    

OrderedDict([('onset', 36.561466), ('duration', 0.23199462890625), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 199.398877197938), ('duration', 0.5705811736459339), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 297.006378), ('duration', 1.343414306640625), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 598.375549), ('duration', 1.24615478515625), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 912.2122378363111), ('duration', 1.3219405409222418), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 917.983459), ('duration', 2.0059814453125), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 950.0993926982558), ('duration', 1.180707577148496), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 982.544373), ('duration', 1.3768310546875), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 1007.5428995127463), ('duration', 1.423628

/tmp/ipykernel_9061/3338397663.py:7: RuntimeWarning: This filename (../ieeg/saved_annotations.fif) does not conform to MNE naming conventions. All annotations files should end with -annot.fif, -annot.fif.gz, _annot.fif, _annot.fif.gz, .txt or .csv
  filtered_notch_data.annotations.save('../ieeg/saved_annotations.fif', overwrite = True)


## Save Files

In [14]:
# save filtered data #
filtered_notch_data.save(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif", overwrite = True)


# save raw clean data #
raw_fif.save(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif", overwrite = True)


Writing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_clean_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_clean_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_clean_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_clean_ieeg.fif
[done]
Writing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_clean_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_clean_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_clean_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_clean_ieeg.fif
[done]
